In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# Set up the data directory
data_dir = 'Dataset/train/'

# Set up the hyperparameters
batch_size = 32
image_size = (224, 224)
learning_rate = 0.001
num_epochs = 10
num_classes = 15

# Create the data generator
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use the training subset of the data
)

validation_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use the validation subset of the data
)


Found 5817 images belonging to 15 classes.
Found 1445 images belonging to 15 classes.


In [7]:
# Create the base model
base_model = MobileNetV2(
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
# Train the model
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator
)

Epoch 1/10
182/182 [==============================] - 143s 764ms/step - loss: 0.4941 - accuracy: 0.8534 - val_loss: 0.3413 - val_accuracy: 0.8983
Epoch 2/10
182/182 [==============================] - 139s 763ms/step - loss: 0.2570 - accuracy: 0.9168 - val_loss: 0.3620 - val_accuracy: 0.8879
Epoch 3/10
182/182 [==============================] - 138s 760ms/step - loss: 0.2236 - accuracy: 0.9276 - val_loss: 0.3050 - val_accuracy: 0.9121
Epoch 4/10
182/182 [==============================] - 138s 760ms/step - loss: 0.1954 - accuracy: 0.9376 - val_loss: 0.3192 - val_accuracy: 0.9017
Epoch 5/10
182/182 [==============================] - 138s 760ms/step - loss: 0.1724 - accuracy: 0.9438 - val_loss: 0.2772 - val_accuracy: 0.9211
Epoch 6/10
182/182 [==============================] - 147s 807ms/step - loss: 0.1335 - accuracy: 0.9579 - val_loss: 0.2715 - val_accuracy: 0.9211
Epoch 7/10
182/182 [==============================] - 214s 1s/step - loss: 0.1315 - accuracy: 0.9534 - val_loss: 0.3236 - va

In [9]:
# Save the trained model
model.save('animal_recognition_model.h5')